In [ ]:
%matplotlib inline

In [11]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates
from datetime import datetime
import sqlite3

project_name = 'mongo'

con = sqlite3.connect('main.db')

sql = f'''
select tag, date(creator_dt)
from projects join releases on projects.id = releases.project_id
where name = '{project_name}'
order by 2
'''
raw_releases = con.execute(sql).fetchall()

import re
from dataclasses import dataclass, field
majormin_pat = re.compile(r'[0-9]+\.[0-9]+')
@dataclass
class Release:
    raw_label: str
    majormin: str = field(init=False)
    
    def __post_init__(self):
        self.majormin = None
        if match := majormin_pat.search(self.raw_label):
            self.majormin = match.group(0)

raw_releases = [Release(tag) for tag,_ in raw_releases]
print(f'{len(raw_releases)=}')
assert 0, [raw_releases[:3]]
    # dates = [datetime.strptime(d, "%Y-%m-%d") for d in dates]


len(raw_releases)=731


AssertionError: [[Release(raw_label='r0.0.3', majormin='0.0'), Release(raw_label='r0.0.4_rc1', majormin='0.0'), Release(raw_label='r0.0.4_rc2', majormin='0.0')]]

In [ ]:
# Choose some nice levels
levels = np.tile([-5, 5, -3, 3, -1, 1],
                 int(np.ceil(len(dates)/6)))[:len(dates)]

# Create figure and plot a stem plot with the date
fig, ax = plt.subplots(figsize=(8.8, 4), constrained_layout=True)
ax.set(title="Matplotlib release dates")

ax.vlines(dates, 0, levels, color="tab:red")  # The vertical stems.
ax.plot(dates, np.zeros_like(dates), "-o",
        color="k", markerfacecolor="w")  # Baseline and markers on it.

# annotate lines
for d, l, r in zip(dates, levels, names):
    ax.annotate(r, xy=(d, l),
                xytext=(-3, np.sign(l)*3), textcoords="offset points",
                horizontalalignment="right",
                verticalalignment="bottom" if l > 0 else "top")

# format xaxis with 4 month intervals
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=4))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%b %Y"))
plt.setp(ax.get_xticklabels(), rotation=30, ha="right")

# remove y axis and spines
ax.yaxis.set_visible(False)
ax.spines[["left", "top", "right"]].set_visible(False)

ax.margins(y=0.1)
plt.show()

.. admonition:: References

   The use of the following functions, methods, classes and modules is shown
   in this example:

   - `matplotlib.axes.Axes.annotate`
   - `matplotlib.axes.Axes.vlines`
   - `matplotlib.axis.Axis.set_major_locator`
   - `matplotlib.axis.Axis.set_major_formatter`
   - `matplotlib.dates.MonthLocator`
   - `matplotlib.dates.DateFormatter`

